# Monitoraggio della Reputazione Aziendale con Analisi del Sentiment

Questo notebook implementa un flusso end-to-end minimo per:
- Caricare un modello di sentiment da Hugging Face: `cardiffnlp/twitter-roberta-base-sentiment-latest`.
- Eseguire inferenza su un dataset pubblico e su testi liberi.
- Calcolare metriche semplici (accuracy, F1).
- Fornire indicazioni per pipeline CI/CD e monitoraggio continuo.

Repository GitHub del progetto: [company_reputation_monitoring](https://github.com/gab-25/company_reputation_monitoring)

## 1. Setup ambiente
Installa le dipendenze necessarie.

In [ ]:
!pip -q install transformers torch datasets evaluate accelerate pandas numpy scikit-learn tqdm

## 2. Import e configurazione
Carichiamo le librerie principali e impostiamo il dispositivo (CPU/GPU).

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from typing import List

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

/home/gab25/Projects/company_reputation_monitoring/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cpu'

## 3. Caricamento modello Hugging Face
Useremo il modello indicato nel README: `cardiffnlp/twitter-roberta-base-sentiment-latest`.

In [2]:
MODEL_ID = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)
model.to(device)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0 if device=='cuda' else -1, truncation=True)
pipe('I love this product!')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'label': 'positive', 'score': 0.9848045110702515}]

## 4. Dataset di esempio
Per una demo rapida, utilizziamo `tweet_eval` (task sentiment) da Hugging Face Datasets. Questo dataset ha label: 0=negative, 1=neutral, 2=positive.

In [3]:
ds = load_dataset('tweet_eval', 'sentiment')
ds

Generating validation split: 100%|██████████| 2000/2000 [00:00<00:00, 747714.41 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

### Mappatura etichette del modello
Il modello CardiffNLP etichetta in ordine [negative, neutral, positive]. Allineiamo le predizioni a 0/1/2 per confronto con il dataset.

In [4]:
id2label = model.config.id2label
label2id = {v:k for k,v in id2label.items()}
label2id

{'negative': 0, 'neutral': 1, 'positive': 2}

## 5. Valutazione rapida
Calcoliamo accuracy ed F1 sul validation (o test) del dataset per una valutazione veloce.

In [5]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm.auto import tqdm

def predict_labels(texts: List[str]) -> List[int]:
    preds = pipe(texts, truncation=True)
    # preds è una lista di dict con 'label' e 'score'
    # Mappiamo 'LABEL_0'/'negative' etc a 0/1/2
    out = []
    for p in preds:
        lab = p['label']
        # Alcuni pipeline restituiscono 'LABEL_X', altri stringhe. Normalizziamo.
        if lab in label2id:
            out.append(label2id[lab])
        else:
            # gestisce 'LABEL_0' stile Auto
            if lab.startswith('LABEL_'):
                out.append(int(lab.split('_')[-1]))
            else:
                # fallback: negative->0, neutral->1, positive->2
                lab_lower = lab.lower()
                if 'neg' in lab_lower:
                    out.append(0)
                elif 'neu' in lab_lower:
                    out.append(1)
                else:
                    out.append(2)
    return out

eval_split = ds.get('validation', ds['test']) if 'validation' in ds else ds['test']
texts = eval_split['text'][:200]  # sottocampione per velocità
y_true = eval_split['label'][:200]
y_pred = predict_labels(texts)

acc = accuracy_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average='macro')
print({'accuracy': acc, 'f1_macro': f1_macro})
print(classification_report(y_true, y_pred, digits=3))

{'accuracy': 0.76, 'f1_macro': 0.7543497371083578}
              precision    recall  f1-score   support

           0      0.650     0.812     0.722        32
           1      0.727     0.744     0.736        86
           2      0.861     0.756     0.805        82

    accuracy                          0.760       200
   macro avg      0.746     0.771     0.754       200
weighted avg      0.770     0.760     0.762       200



## 6. Inferenza su testi liberi
Eseguiamo batch prediction su una lista di frasi esempio e visualizziamo le probabilità.

In [6]:
samples = [
    'Il prodotto è fantastico, super consigliato!',
    'Servizio clienti lento e poco utile.',
    'Esperienza nella media, niente di speciale.'
]
preds = pipe(samples, return_all_scores=True)
for s, pr in zip(samples, preds):
    scores = {p['label']: round(p['score'], 3) for p in pr}
    print(s, '->', scores)

Il prodotto è fantastico, super consigliato! -> {'negative': 0.006, 'neutral': 0.016, 'positive': 0.978}
Servizio clienti lento e poco utile. -> {'negative': 0.025, 'neutral': 0.884, 'positive': 0.091}
Esperienza nella media, niente di speciale. -> {'negative': 0.039, 'neutral': 0.843, 'positive': 0.117}


/home/gab25/Projects/company_reputation_monitoring/.venv/lib/python3.13/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


## 7. Salvataggio risultati
Esempio di salvataggio delle predizioni su CSV, utile per audit o monitoraggio.

In [ ]:
out_df = pd.DataFrame({
    'text': texts,
    'y_true': y_true,
    'y_pred': y_pred
})
out_path = 'sentiment_eval_sample.csv'
out_df.to_csv(out_path, index=False)
out_path